In [17]:
import os
import json
import torch
import torchvision
import numpy as np
from PIL import Image
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import DataLoader
from dataset import CustomDataset

from tqdm import tqdm



In [ ]:

train_dataset = CustomDataset("train_annotations.json")
val_dataset = CustomDataset("val_annotations.json")

train_loader = DataLoader(train_dataset, batch_size=6, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(val_dataset, batch_size=6, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))


In [18]:
def get_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

num_classes = 5  # 4 sınıf + background
model = get_model(num_classes)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)


c:\Users\bilgi\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\bilgi\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:
params = [p for p in model.parameters() if p.requires_grad]

# Daha stabil ve küçük adımlarla öğrenen yapı
optimizer = torch.optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0001)

# Öğrenme oranını her 10 epoch'ta sert bir şekilde düşür
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.3)




In [ ]:
def train_one_epoch(model, optimizer, data_loader, device, epoch):
    model.train()
    running_loss = 0.0

    progress_bar = tqdm(enumerate(data_loader), total=len(data_loader), desc=f"Epoch {epoch + 1}")
    for i, (images, targets) in progress_bar:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        running_loss += losses.item()

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        progress_bar.set_postfix({
            "total_loss": f"{losses.item():.4f}",
            "cls_loss": f"{loss_dict['loss_classifier'].item():.4f}",
            "box_loss": f"{loss_dict['loss_box_reg'].item():.4f}"
        })

    avg_loss = running_loss / len(data_loader)
    print(f"🔵 Epoch [{epoch + 1}] tamamlandı. Ortalama Loss: {avg_loss:.4f}")


In [ ]:
def evaluate_map(model, data_loader, device):
    model.eval()
    metric = MeanAveragePrecision(iou_type="bbox")  # tüm IoU aralıklarını içerir

    with torch.no_grad():
        for images, targets in data_loader:
            images = [img.to(device) for img in images]
            outputs = model(images)

            # ground truth ve prediction formatlarını hazırla
            ground_truths = []
            predictions = []

            for target in targets:
                gt = {
                    "boxes": target["boxes"].cpu(),
                    "labels": target["labels"].cpu()
                }
                ground_truths.append(gt)

            for output in outputs:
                pred = {
                    "boxes": output["boxes"].cpu(),
                    "scores": output["scores"].cpu(),
                    "labels": output["labels"].cpu()
                }
                predictions.append(pred)

            metric.update(predictions, ground_truths)

    result = metric.compute()
    print("\n📊 Değerlendirme Sonuçları:")
    print(f"🔸 mAP@50: {result['map_50']:.4f}")
    print(f"🔸 mAP@75: {result['map_75']:.4f}")
    print(f"🔸 mAP (IoU 0.5:0.95): {result['map']:.4f}")
    print(f"🔸 mAR: {result['mar_100']:.4f}")
    return result




In [ ]:
num_epochs = 50

for epoch in range(num_epochs):
    print(f"\n🟢 Epoch [{epoch + 1}/{num_epochs}] başlıyor...")
    train_one_epoch(model, optimizer, train_loader, device, epoch)
    lr_scheduler.step()

    if (epoch + 1) % 5 == 0:
        print(f"\n🧪 Değerlendirme (Epoch {epoch + 1})")
        evaluate_map(model, val_loader, device)

    model_path = f"fasterrcnn_epoch_{epoch + 1}.pth"
    torch.save(model.state_dict(), model_path)
    print(f"💾 Model saved: {model_path}")

    


🚀 Epoch [1/40] başlıyor...


PicklingError: Can't pickle <function <lambda> at 0x000002253B974860>: attribute lookup <lambda> on __main__ failed